In [10]:
from transformers import SpeechT5Processor, SpeechT5ForSpeechToSpeech
from transformers import SpeechT5HifiGan

from datasets import load_dataset

import torch

from IPython.display import Audio
import soundfile as sf

In [2]:
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_vc")
model = SpeechT5ForSpeechToSpeech.from_pretrained("microsoft/speecht5_vc")

In [4]:
dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
example = dataset[40]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset librispeech_asr_demo downloaded and prepared to /export/home/lium/bdos/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_demo/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b. Subsequent calls will reuse this data.


In [15]:
example["audio"]

{'path': '/export/home/lium/bdos/.cache/huggingface/datasets/downloads/extracted/e1169fd1dddcf165665bd3c40c38a3f2e7f677cf94c6387cfa40bdaf65d475f6/dev_clean/1272/141231/1272-141231-0000.flac',
 'array': array([-0.00048828, -0.00018311, -0.00137329, ...,  0.00079346,
         0.00091553,  0.00085449]),
 'sampling_rate': 16000}

In [16]:
Audio(example["audio"]['array'], rate=16000)

In [5]:
sampling_rate = dataset.features["audio"].sampling_rate
inputs = processor(audio=example["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")

In [7]:
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

Found cached dataset cmu-arctic-xvectors (/export/home/lium/bdos/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/a62fea1f9415e240301ea0042ffad2a3aadf4d1caa7f9a8d9512d631723e781f)


In [9]:
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [11]:
speech = model.generate_speech(inputs["input_values"], speaker_embeddings, vocoder=vocoder)

In [12]:
Audio(speech, rate=16000)

In [13]:
#sf.write("speech_converted.wav", speech.numpy(), samplerate=16000)